# Redis Hashes Tutorial

## Setup

In [1]:
%%capture
%pip install redis rich tqdm 

In [2]:
import redis
from rich.pretty import pprint

In [3]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

## Redis Hashes

Lists, sets, and sorted sets are great for many use cases, but the hash data type is on a higher level. Redis hashes are maps (sequences of field-value pairs) and are used to represent objects in Redis.

Redis hashes are record types that are structured as name-value pairs. Consider the following example that shows how to create a hash key using the `HSET` command.

In [4]:
# Create a hash
num_fields_added = r.hset("bike:1", 
                           mapping={
                               "model": "Deimos",
                               "brand": "Ergonom",
                               "type": "Enduro bikes",
                               "price": 4972
                           })
print(f"Number of fields added: {num_fields_added}")

Number of fields added: 4


`HSET` returns the number of added name-value pairs.

To retrieve the stored data, use the `HGETALL` command.

In [5]:
# HGETALL usage
# returns all the name-value pairs associated with the key
all_fields = r.hgetall("bike:1")
pprint(all_fields)

{b'model': b'Deimos', b'brand': b'Ergonom', b'type': b'Enduro bikes', b'price': b'4972'}

If you only want the values of a subset of the fields, use the `HGET` command.

In [6]:
# HGET usage
price = r.hget("bike:1", "price")
print(f"The price of the bike is: {price}")

The price of the bike is: b'4972'


You can update fields in a hash using `HSET` by specifying a subset of its name-value pairs.

In [9]:
# Update an existing hash
num_created = r.hset("bike:1", 
                      mapping={
                          "model": "Kraken",
                          "price": 3000
                      })
print(f"Number of new fields: {num_created}")

# Let's check the updated hash
updated_bike = r.hgetall("bike:1")
pprint(updated_bike)

Number of new fields: 0


{b'model': b'Kraken', b'brand': b'Ergonom', b'type': b'Enduro bikes', b'price': b'3000'}

Use the `HDEL` command to delete one or more fields from a hash. Once all fields are removed, the hash key itself will be deleted.

In [10]:
# Delete hash fields and keys
deleted = r.hdel("bike:1", "model")
print(f"Number of fields deleted: {deleted}")
print("Hash after deleting model field:")
pprint(r.hgetall("bike:1"))

# Delete remaining fields
deleted = r.hdel("bike:1", "brand", "type", "price")
print(f"\nNumber of fields deleted: {deleted}")
print("Hash after deleting all fields:")
pprint(r.hgetall("bike:1"))

# Check if the key still exists
exists = r.exists("bike:1")
print(f"\nDoes key bike:1 still exist? {exists}")

Number of fields deleted: 1
Hash after deleting model field:


{b'brand': b'Ergonom', b'type': b'Enduro bikes', b'price': b'3000'}


Number of fields deleted: 3
Hash after deleting all fields:


{}


Does key bike:1 still exist? 0


Integer values in hash keys can be incremented or decremented in the same way as simple string keys using the `HINCRBY` command.
The increment value must be a positive or negative integer.

In [11]:
# First let's create a new hash with a price field
r.hset("bike:1", mapping={"price": 1000})
print(f"Initial price: {r.hget('bike:1', 'price')}")

# Hash INCRBY usage
new_price = r.hincrby("bike:1", "price", 100)
print(f"Price after incrementing by 100: {new_price}")

new_price = r.hincrby("bike:1", "price", -100)
print(f"Price after decrementing by 100: {new_price}")

Initial price: b'1000'
Price after incrementing by 100: 1100
Price after decrementing by 100: 1000


New in [Redis Community Edition version 7.4](https://hub.docker.com/layers/redis/redis-stack/7.4.0-v0/images/sha256-3e3c86603a81712d1311bc619ad124de15b2dca2b50722f23a4502b4d4054ba8) is the ability set the expiration time or the remaining time-to-live (TTL) for individual hash fields. This is called hash field expiration (HFE). HFE works just like [key expiration](https://redis.io/docs/latest/develop/use/keyspace/?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials#key-expiration) and includes the following commands:

- `hexpire` - set an expiration (time-to-live or TTL) in seconds on a hash key's field(s).
- `hexpireat` - set a TTL as an absolute Unix timestamp (seconds since Unix epoch) on a hash key's field(s).
- `hexpiretime` - returns the absolute Unix timestamp (seconds since Unix epoch) at which the hash key's field(s) will expire.
- `hpersist` - remove expirations from a hash key's field(s).
- `hpexpire` - set an expiration (time-to-live or TTL) in milliseconds on a hash key's field(s).
- `hpexpireat` - set a TTL as an absolute Unix timestamp (milliseconds since Unix epoch) on a hash key's field(s).
- `hpexpiretime` - returns the absolute Unix timestamp (milliseconds since Unix epoch) at which the the hash key's field(s) will expire.
- `hpttl` - returns the remaining TTL in milliseconds of a hash key's field(s).
- `httl` - returns the remaining TTL in seconds of a hash key's field(s).

**Note**: Hash Field Expiration (HFE) commands may not be available in all versions of the Python `redis` client. HFE features were introduced in Redis 7.4 and may require the use of custom commands via the `execute_command` method or an updated version of the client library. If the commands don't work directly, you can use the following approach.

In [14]:
# HFE example 1
# Create hash
r.hset("hash", mapping={
    "field1": "foo",
    "field2": "bar",
    "field3": "baz"
})
print("Hash created:")
pprint(r.hgetall("hash"))

# Using execute_command for HEXPIRE
try:
    # Try with direct method 
    result = r.hexpire("hash", 10,"field2")
    print(f"\nHEXPIRE result: {result}")
    
    # Check TTL 
    ttl = r.httl("hash", "field2")
    print(f"TTL of field2: {ttl} seconds")
except Exception as e:
    print(f"\nNote: HFE commands might not be supported in your Redis version: {e}")

Hash created:


{b'field1': b'foo', b'field3': b'baz', b'field2': b'bar'}


HEXPIRE result: [1]
TTL of field2: [10] seconds


In [15]:
# HFE example 1 cont'd
try:
    # Check TTL after a few seconds
    ttl = r.httl("hash","field2")
    print(f"TTL of field2 after a few seconds: {ttl} seconds")
    
    # Check complete hash
    print("\nComplete hash:")
    pprint(r.hgetall("hash"))
except Exception as e:
    print(f"\nNote: HFE commands might not be supported in your Redis version: {e}")

TTL of field2 after a few seconds: [-2] seconds

Complete hash:


{b'field1': b'foo', b'field3': b'baz'}

In [16]:
# HFE example 2
# Create a new hash for example 2
r.hset("hash2", mapping={
    "field1": "foo",
    "field2": "bar",
    "field3": "baz"
})
print("Hash2 created:")
pprint(r.hgetall("hash2"))

try:
    # Set expiration on field1 and field3
    result = r.hexpire("hash2", 10, "field1", "field3")
    print(f"\nHEXPIRE result for field1 and field3: {result}")
    
    # Check TTL
    ttl = r.httl("hash2","field1", "field3")
    print(f"TTL of field1 and field3: {ttl} seconds")
    
    # Remove expiration from field3
    result = r.hpersist("hash2", "field3")
    print(f"\nHPERSIST result for field3: {result}")
    
    # Check TTL again
    ttl = r.httl( "hash2", "field1", "field3")
    print(f"Updated TTL of field1 and field3: {ttl} seconds")
except Exception as e:
    print(f"\nNote: HFE commands might not be supported in your Redis version: {e}")

Hash2 created:


{b'field1': b'foo', b'field2': b'bar', b'field3': b'baz'}


HEXPIRE result for field1 and field3: [1, 1]
TTL of field1 and field3: [10, 10] seconds

HPERSIST result for field3: [1]
Updated TTL of field1 and field3: [10, -1] seconds


In [17]:
# HFE example 2 cont'd
try:
    # Check the state of the hash after a few seconds
    print("\nHash2 after TTL management:")
    pprint(r.hgetall("hash2"))
except Exception as e:
    print(f"\nNote: HFE commands might not be supported in your Redis version: {e}")


Hash2 after TTL management:


{b'field2': b'bar', b'field3': b'baz'}

## Resources

- Hash type [reference page](https://redis.io/docs/data-types/hashes?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Entire set of [Redis hash commands](https://redis.io/commands/?group=hash?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Check out [Get started with Redis](https://university.redis.io/learningpath/14q8m6gilfwltm?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials) learning path on Redis University for an introduction to working with all core data structures in Redis.